<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#라이브러리-설치-및-데이터-로드" data-toc-modified-id="라이브러리-설치-및-데이터-로드-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>라이브러리 설치 및 데이터 로드</a></span></li><li><span><a href="#전처리-함수-정의" data-toc-modified-id="전처리-함수-정의-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>전처리 함수 정의</a></span></li><li><span><a href="#Trian-Data에-대해-전처리-수행" data-toc-modified-id="Trian-Data에-대해-전처리-수행-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Trian Data에 대해 전처리 수행</a></span></li><li><span><a href="#Train-Data에-대해-검증" data-toc-modified-id="Train-Data에-대해-검증-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train Data에 대해 검증</a></span></li><li><span><a href="#전처리-완료-문장-랜덤-확인" data-toc-modified-id="전처리-완료-문장-랜덤-확인-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>전처리 완료 문장 랜덤 확인</a></span></li><li><span><a href="#전처리가-되지-않은-문장-랜덤-확인" data-toc-modified-id="전처리가-되지-않은-문장-랜덤-확인-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>전처리가 되지 않은 문장 랜덤 확인</a></span></li><li><span><a href="#Test-Data에-대한-전처리-수행-및-검증" data-toc-modified-id="Test-Data에-대한-전처리-수행-및-검증-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Test Data에 대한 전처리 수행 및 검증</a></span></li><li><span><a href="#전처리가-완료된-문장-랜덤-확인" data-toc-modified-id="전처리가-완료된-문장-랜덤-확인-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>전처리가 완료된 문장 랜덤 확인</a></span></li><li><span><a href="#전처리가-적용되지-않은-문장-랜덤-확인" data-toc-modified-id="전처리가-적용되지-않은-문장-랜덤-확인-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>전처리가 적용되지 않은 문장 랜덤 확인</a></span></li></ul></div>

# 라이브러리 설치 및 데이터 로드

- train data와 test data가 있는 경로를 직접 설정해주세요.

In [128]:
import re
import pandas as pd
from tqdm.notebook import tqdm
import random

# train 데이터, test 데이터를 로드
train_df = pd.read_csv('./train_data.csv')
test_df = pd.read_csv('./test_data.csv')

# 한 개의 cell 내용이 잘리지 않고 모두 보이도록 설정
pd.set_option('display.max_colwidth', None)

# 전처리 함수 정의
- **git에 push한 코드와 다른 점: 전처리 진행 상황을 보기 위해 line 85에서 tqdm을 적용**
- 추후 실제 코드에서도 적용 예정

In [104]:
def fix_entity_index(idx: int, sentence: str, subject_entity: str, object_entity: str) -> tuple:
    dict_subject = eval(subject_entity)
    dict_object = eval(object_entity)
    
    s_word, s_start, s_end, s_typ = dict_subject.values()
    o_word, o_start, o_end, o_typ = dict_object.values()
    
    matched = re.finditer(re.escape(s_word), sentence)
    new_standard = float("inf")
    for word in matched:
        start_standard = abs(s_start - word.span()[0])
        if new_standard >= start_standard:
            s, e = word.span()[0], word.span()[1] - 1
            new_standard = start_standard
    dict_subject['start_idx'], dict_subject['end_idx']= s, e

    matched = re.finditer(re.escape(o_word), sentence)
    new_standard = float("inf")
    for word in matched:
        start_standard = abs(o_start - word.span()[0])
        if new_standard >= start_standard:
            s, e = word.span()[0], word.span()[1] - 1
            new_standard = start_standard
    dict_object['start_idx'], dict_object['end_idx']= s, e
    
    dict_subject = str(dict_subject)
    dict_object = str(dict_object)
    
    return (dict_subject, dict_object)

def preprocess_overlapped_bracket(sts: str) -> str:
    result = sts
    matched_list = re.finditer('\([^\(\)]+\)', result)
    idx = ()
    word = ''
    for match in matched_list:
        idx_2 = match.span()
        word_2 = match.group()
        if word == word_2 and idx[1] == idx_2[0]:
            result = result.replace(word * 2, word)
        idx = idx_2
        word = word_2
    return result

def preprocess_double_quotation(sts: str) -> str:
    result = sts
    matched_list = re.findall('"".*?""', result)
    for word in matched_list:
        result = result.replace(word, word[1:-1])
    matched_list = re.findall('\(""\)', result)
    for word in matched_list:
        result = result.replace(word, '')
    matched_list = re.findall('""', result)
    for word in matched_list:
        result = result.replace(word, '')
    matched_list = re.findall('  ', result)
    for word in matched_list:
        result = result.replace(word, ' ')
    return result

def preprocess_double_sqbracket(sts: str) -> str:
    result = sts
    result = result.replace('[[', '[')
    result = result.replace(']]', ']')
    return result

def preprocess_spaced_and_double_comma(sts: str) -> str:
    result = sts
    result = result.replace(" , ", ", ")
    result = result.replace(",,", ",")
    return result

def preprocess_double_dash(sts: str) -> str:
    result = sts
    result = result.replace("--", "-")
    return result

def preprocess_double_space(sts: str) -> str:
    result = sts
    result = result.replace("  ", " ")
    return result

def data_cleansing(input_df: pd.DataFrame) -> pd.DataFrame:
    df = input_df.copy()
    for row in tqdm(df.iterrows(), total = len(df)):
        idx, sentence, subject_entity, object_entity = row[1].id, row[1].sentence, row[1].subject_entity, row[1].object_entity
        sentence = preprocess_double_quotation(sentence)
        sentence = preprocess_double_sqbracket(sentence)
        sentence = preprocess_spaced_and_double_comma(sentence)
        sentence = preprocess_double_dash(sentence)
        sentence = preprocess_overlapped_bracket(sentence)
        sentence = preprocess_double_space(sentence)
        subject_entity, object_entity = fix_entity_index(idx, sentence, subject_entity, object_entity)
        df.loc[idx, 'sentence'] = sentence
        df.loc[idx, 'subject_entity'] = subject_entity
        df.loc[idx, 'object_entity'] = object_entity
    return df

# Trian Data에 대해 전처리 수행

In [35]:
#전처리 수행 후 processed_train_df에 저장
processed_train_df = data_cleansing(train_df)
# 혼동을 막기 위해 전처리가 완료된 데이터셋에는 "2"를 붙여 column 이름 수정
processed_train_df.columns = ['id2', 'sentence2', 'subject_entity2', 'object_entity2', 'label2', 'source2']

  0%|          | 0/32470 [00:00<?, ?it/s]

C:\Users\forma\AppData\Local\Temp/ipykernel_23388/1117533516.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  processed_train_df.colums = ['id2','sentence2','subject_entity2','object_entity2','label','source']


# Train Data에 대해 검증

- **원본 데이터셋 문장과 전처리한 데이터셋 문장이 서로 다른 id를 찾습니다.**
- 두 데이터셋에서 동일한 id에 대해 문장이 서로 다르다면 전처리가 적용된 문장이라고 할 수 있습니다.
- 이것은 밑의 코드에서 ```train_df['sentence'] != processed_train_df['sentence2']``` 부분입니다.
---
- **문장이 서로 다른 id들을 찾았다면 해당 id의 원본 문장과 전처리 후 문장을 서로 확인할 수 있어야 합니다.**
- 원본 문장을 ```org_list_sentence_changed``` 에 저장하고, 전처리 후 문장을 ```prc_list_sentence_changed```에 저장합니다.

In [117]:
# 원본 문장을 org_list_sentence_changed에 저장
org_list_sentence_changed = train_df[train_df['sentence'] != processed_train_df['sentence2']]
# 전처리 완료 문장을 prc_list_sentence_changed에 저장
prc_list_sentence_changed = processed_train_df[train_df['sentence'] != processed_train_df['sentence2']]

In [119]:
# 전처리가 완료된 문장의 원본 문장
# 전체 내용을 보고 싶다면 추가로 셀을 만들어 확인하시면 됩니다.
org_list_sentence_changed

,id,sentence,subject_entity,object_entity,label,source
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_idx': 20, 'type': 'DAT'}",per:date_of_birth,wikipedia
35,35,"로버트 조지 (바비) 실(Robert George (Bobby)(Bobby) Seale, 1936년 10월 22일 ~ )은 아프리카계 미국인 인권 운동가이자, 휴이 뉴튼과 함께 흑표당의 창립자이다.","{'word': '흑표당', 'start_idx': 99, 'end_idx': 101, 'type': 'ORG'}","{'word': '휴이 뉴튼', 'start_idx': 89, 'end_idx': 93, 'type': 'PER'}",org:founded_by,wikipedia
41,41,"이 캐릭터가 작가 스탠 리와 만화가 잭 커비가 만든 """" 에서 캡틴 아메리카의 전쟁 시절 애정 상대로 처음으로 등장하였을 당시에는 이름이 없었다.","{'word': '스탠 리', 'start_idx': 10, 'end_idx': 13, 'type': 'PER'}","{'word': '만화가', 'start_idx': 16, 'end_idx': 18, 'type': 'POH'}",no_relation,wikipedia
215,215,"박흥식(朴興植, 1903년 9월 26일(1903년 음력 8월 6일)(1903년 음력 8월 6일) 대한제국 평안남도 용강 출생 - 1994년 5월 10일 대한민국 서울에서 별세.)은 일제 강점기의 대표적인 기업인이다.","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1994년 5월 10일', 'start_idx': 72, 'end_idx': 83, 'type': 'DAT'}",per:date_of_death,wikipedia
535,535,"FC 서울은 창단 후 K리그 참가 첫 시즌인 1984 시즌부터 1985 시즌까지는 주 유니폼과 부 유니폼 구분없이 빨간색 유니폼과 노란색 유니폼을 번갈아 착용하였으며, 1988 시즌부터 1994 시즌까지 노란색 주 유니폼을 , 1995 시즌부터 2004 시즌까지 주로 빨간색 주 유니폼을 착용하였다.","{'word': 'FC 서울', 'start_idx': 0, 'end_idx': 4, 'type': 'ORG'}","{'word': '1985', 'start_idx': 35, 'end_idx': 38, 'type': 'DAT'}",no_relation,wikipedia
...,...,...,...,...,...,...
32011,32011,"괴팅겐 7교수 사건으로 알려진 빌헬름 에두아르트 베버와 그림 형제, 수학에서는 카를 프리드리히 가우스, 베른하르트 리만, 클라인 병으로 알려진 펠릭스 클라인, 다비트 힐베르트, 게르하르트 겐첸 , 헬무트 하세, 헤르만 바일, 철학은 에드문트 후설, 사회학에서는 헬무트 플레스너, 물리학에서는 베르너 하이젠베르크, 막스 플랑크, 법학에서는 루돌프 폰 예링, 신학은 칼 바르트 같은 저명한 학자들이 여기에서 교수직을 맡았다.","{'word': '베르너 하이젠베르크', 'start_idx': 163, 'end_idx': 172, 'type': 'PER'}","{'word': '물리학', 'start_idx': 156, 'end_idx': 158, 'type': 'POH'}",no_relation,wikipedia
32117,32117,"백다은(白多恩, (姜恩惠)(姜恩惠), 1988년 3월 28일 ~ )은 대한민국의 배우이다.","{'word': '백다은', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1988년 3월 28일', 'start_idx': 21, 'end_idx': 32, 'type': 'DAT'}",per:date_of_birth,wikipedia
32153,32153,"방사겸(영어 이름은 크리스토퍼 뱅(Christopher Bang)(Christopher Bang), 1881년 7월 25일 ~ 1955년 12월 8일)은 대한제국 관료 출신 독립운동가, 외교관 겸 정치인이다.","{'word': '방사겸', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1881년 7월 25일', 'start_idx': 56, 'end_idx': 67, 'type': 'DAT'}",per:date_of_birth,wikipedia
32168,32168,"김종서(金宗瑞, 1383년 고려국 양광도 공주 출생 ~ 1453년 11월 10일(음력 10월 10일)(음력 10월 10일) 조선국 한성부에서 암살됨.)는 조선 전기의 문신, 군인, 정치가이다.","{'word': '김종서', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1453년 11월 10일', 'start_idx': 31, 'end_idx': 43, 'type': 'DAT'}",per:date_of_death,wikipedia


In [120]:
# 전처리가 완료된 문장
prc_list_sentence_changed

,id2,sentence2,subject_entity2,object_entity2,label2,source2
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_idx': 20, 'type': 'DAT'}",per:date_of_birth,wikipedia
35,35,"로버트 조지 (바비) 실(Robert George (Bobby) Seale, 1936년 10월 22일 ~ )은 아프리카계 미국인 인권 운동가이자, 휴이 뉴튼과 함께 흑표당의 창립자이다.","{'word': '흑표당', 'start_idx': 92, 'end_idx': 94, 'type': 'ORG'}","{'word': '휴이 뉴튼', 'start_idx': 82, 'end_idx': 86, 'type': 'PER'}",org:founded_by,wikipedia
41,41,이 캐릭터가 작가 스탠 리와 만화가 잭 커비가 만든 에서 캡틴 아메리카의 전쟁 시절 애정 상대로 처음으로 등장하였을 당시에는 이름이 없었다.,"{'word': '스탠 리', 'start_idx': 10, 'end_idx': 13, 'type': 'PER'}","{'word': '만화가', 'start_idx': 16, 'end_idx': 18, 'type': 'POH'}",no_relation,wikipedia
215,215,"박흥식(朴興植, 1903년 9월 26일(1903년 음력 8월 6일) 대한제국 평안남도 용강 출생 - 1994년 5월 10일 대한민국 서울에서 별세.)은 일제 강점기의 대표적인 기업인이다.","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1994년 5월 10일', 'start_idx': 56, 'end_idx': 67, 'type': 'DAT'}",per:date_of_death,wikipedia
535,535,"FC 서울은 창단 후 K리그 참가 첫 시즌인 1984 시즌부터 1985 시즌까지는 주 유니폼과 부 유니폼 구분없이 빨간색 유니폼과 노란색 유니폼을 번갈아 착용하였으며, 1988 시즌부터 1994 시즌까지 노란색 주 유니폼을, 1995 시즌부터 2004 시즌까지 주로 빨간색 주 유니폼을 착용하였다.","{'word': 'FC 서울', 'start_idx': 0, 'end_idx': 4, 'type': 'ORG'}","{'word': '1985', 'start_idx': 35, 'end_idx': 38, 'type': 'DAT'}",no_relation,wikipedia
...,...,...,...,...,...,...
32011,32011,"괴팅겐 7교수 사건으로 알려진 빌헬름 에두아르트 베버와 그림 형제, 수학에서는 카를 프리드리히 가우스, 베른하르트 리만, 클라인 병으로 알려진 펠릭스 클라인, 다비트 힐베르트, 게르하르트 겐첸, 헬무트 하세, 헤르만 바일, 철학은 에드문트 후설, 사회학에서는 헬무트 플레스너, 물리학에서는 베르너 하이젠베르크, 막스 플랑크, 법학에서는 루돌프 폰 예링, 신학은 칼 바르트 같은 저명한 학자들이 여기에서 교수직을 맡았다.","{'word': '베르너 하이젠베르크', 'start_idx': 162, 'end_idx': 171, 'type': 'PER'}","{'word': '물리학', 'start_idx': 155, 'end_idx': 157, 'type': 'POH'}",no_relation,wikipedia
32117,32117,"백다은(白多恩, (姜恩惠), 1988년 3월 28일 ~ )은 대한민국의 배우이다.","{'word': '백다은', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1988년 3월 28일', 'start_idx': 16, 'end_idx': 27, 'type': 'DAT'}",per:date_of_birth,wikipedia
32153,32153,"방사겸(영어 이름은 크리스토퍼 뱅(Christopher Bang), 1881년 7월 25일 ~ 1955년 12월 8일)은 대한제국 관료 출신 독립운동가, 외교관 겸 정치인이다.","{'word': '방사겸', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1881년 7월 25일', 'start_idx': 38, 'end_idx': 49, 'type': 'DAT'}",per:date_of_birth,wikipedia
32168,32168,"김종서(金宗瑞, 1383년 고려국 양광도 공주 출생 ~ 1453년 11월 10일(음력 10월 10일) 조선국 한성부에서 암살됨.)는 조선 전기의 문신, 군인, 정치가이다.","{'word': '김종서', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1453년 11월 10일', 'start_idx': 31, 'end_idx': 43, 'type': 'DAT'}",per:date_of_death,wikipedia


- 총 299개의 문장이 전처리가 완료되었음을 알 수 있습니다.

# 전처리 완료 문장 랜덤 확인

- 다음 셀을 반복 실행해서 전처리가 완료된 문장과 원본 문장을 확인해볼 수 있습니다.

In [173]:
random_id = random.choice(list(prc_list_sentence_changed.id2))
print("1. original id: ", org_list_sentence_changed.loc[random_id].id)
print("2. original sentence: ", org_list_sentence_changed.loc[random_id].sentence)
print("3. original subject_entity: ", org_list_sentence_changed.loc[random_id].subject_entity)
print("4. original object_entity: ", org_list_sentence_changed.loc[random_id].object_entity)
print("5. original label: ", org_list_sentence_changed.loc[random_id].label)
print("6. original source: ", org_list_sentence_changed.loc[random_id].source)
print("-"*120)
print("1. preprocessed id: ", prc_list_sentence_changed.loc[random_id].id2)
print("2. preprocessed sentence: ", prc_list_sentence_changed.loc[random_id].sentence2)
print("3. preprocessed subject_entity: ", prc_list_sentence_changed.loc[random_id].subject_entity2)
print("4. preprocessed object_entity: ", prc_list_sentence_changed.loc[random_id].object_entity2)
print("5. preprocessed label: ", prc_list_sentence_changed.loc[random_id].label2)
print("6. preprocessed source: ", prc_list_sentence_changed.loc[random_id].source2)

1. original id:  26633
2. original sentence:  1931년 봉천(현재의 심양) 근교에서 만주사변의 시작을 알리는 유조호 사건1)이 일어났고, 이를 발단으로 관동군 방역부가 정식 발족되었다. 또한 말이나 동물에 대한 생물전을 연구하는 부대인 관동군 군마공역창(100부대)도 현재의 장춘 시에 설치되었다. 731부대 발족 후 1938년부터 1942년 사이에 북경, 남경, 광동(현 광저우 , 싱가포르에 731부대의 자매부대 격인 방역급수가 만들어졌다.
3. original subject_entity:  {'word': '731부대', 'start_idx': 142, 'end_idx': 146, 'type': 'ORG'}
4. original object_entity:  {'word': '관동군', 'start_idx': 60, 'end_idx': 62, 'type': 'ORG'}
5. original label:  no_relation
6. original source:  wikipedia
------------------------------------------------------------------------------------------------------------------------
1. preprocessed id:  26633
2. preprocessed sentence:  1931년 봉천(현재의 심양) 근교에서 만주사변의 시작을 알리는 유조호 사건1)이 일어났고, 이를 발단으로 관동군 방역부가 정식 발족되었다. 또한 말이나 동물에 대한 생물전을 연구하는 부대인 관동군 군마공역창(100부대)도 현재의 장춘 시에 설치되었다. 731부대 발족 후 1938년부터 1942년 사이에 북경, 남경, 광동(현 광저우, 싱가포르에 731부대의 자매부대 격인 방역급수가 만들어졌다.
3. preprocessed subject_entity:  {'word': '731부대', 'start_idx': 142, 'end_idx': 14

---

- **다음은 전처리가 완료된 문장의 subject_entity, object_entity의 index가 올바르게 변경되었는지 확인합니다.**
- ```prc_list_sentence_changed```가 변경된 문장들을 담고 있는 데이터셋이었으니 해당 데이터셋을 사용하여 index가 올바르게 변경되었는지 확인합니다.
---

In [91]:
count = 0

# 해당 데이터셋의 row를 한 개씩 방문합니다. 
for row in tqdm(prc_list_sentence_changed.iterrows(), total = len(prc_list_sentence_changed)):
    
#   id, sentence, subject_entity, object_entity를
#   각각 idx, sts, subject_dict, object_dict로 저장합니다.
    idx = row[1].id2
    sts, subject_dict, object_dict = row[1].sentence2, eval(row[1].subject_entity2), eval(row[1].object_entity2)
    
#   subject_dict에서 target 단어, 단어의 시작 인덱스, 단어의 끝 인덱스를 각각 s_word, s_start, s_end로 저장합니다. 
#   이 때 각 인덱스는 전처리로 인해 변경되었을 수도 되지 않았을 수도 있습니다.
#   (전처리로 인해 삭제된 부분이 entity 단어 뒤에 나온 경우 index 위치는 전처리에 의해 영향을 받지 않기 때문에 
#   변경되지 않고 그대로 남아있을 수 있습니다)
    s_word, s_start, s_end = subject_dict['word'], subject_dict['start_idx'], subject_dict['end_idx']
    
#   object_dict에서 target 단어, 단어의 시작 인덱스, 단어의 끝 인덱스를 각각 o_word, o_start, o_end로 저장합니다.
    o_word, o_start, o_end = object_dict['word'], object_dict['start_idx'], object_dict['end_idx']
    
#   변경된 문장에서 s_start와 s_end의 범위에 해당하는 부분이 s_word와 일치하는지 확인합니다. 
#   일치하지 않다면, 전처리로 인해 변경이 잘못된 경우입니다. 
#   이 경우 count를 증가시키고 잘못 수정한 문장의 id를 출력하도록 합니다.
    if sts[s_start:s_end + 1] != s_word:
        count += 1
        print("wrong subject idx: " + idx)
        
#   object_entity에 대해서도 동일하게 적용합니다.
    if sts[o_start:o_end + 1] != o_word:
        count += 1
        print("wrong object idx: " + idx)
        
# 전처리가 된 문장들의 entity index가 모두 올바르게 적용이 되었다면 count의 값은 0이므로 "perfect preprocessing!"을 출력합니다.
if count == 0:
    print("perfect preprocessing!")

  0%|          | 0/299 [00:00<?, ?it/s]

perfect preprocessing!


---

- **원본 문장과 전처리 이후 문장이 서로 같은 id들을 찾습니다.**
- **두 문장이 서로 같다면 전처리가 필요없는 문장이었음을 의미합니다.**

In [121]:
org_list_sentence_unchanged = train_df[train_df['sentence'] == processed_train_df['sentence2']]
prc_list_sentence_unchanged = processed_train_df[train_df['sentence'] == processed_train_df['sentence2']]

In [122]:
org_list_sentence_unchanged

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': 23, 'type': 'ORG'}","{'word': '대안신당', 'start_idx': 14, 'end_idx': 17, 'type': 'ORG'}",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 24, 'type': 'ORG'}","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx': 41, 'type': 'ORG'}",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': 17, 'type': 'ORG'}","{'word': '박정부', 'start_idx': 22, 'end_idx': 24, 'type': 'PER'}",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_idx': 30, 'type': 'ORG'}","{'word': '1967', 'start_idx': 0, 'end_idx': 3, 'type': 'DAT'}",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등록 마감 결과, 강석호(3선·경북 영양·영덕·봉화·울진)-이장우(재선·대전 동구), 유기준(4선·부산 서구·동구)-박성중(초선·서울 서초을), 김선동(재선·서울 도봉을)-김종석(초선·비례), 심재철(5선·경기 안양시동안구을)-김재원(3선·경북 상주·군위·의성·청송) 등 4개 조가 등록했다고 밝혔다.","{'word': '유기준', 'start_idx': 93, 'end_idx': 95, 'type': 'PER'}","{'word': '부산 서구·동구', 'start_idx': 100, 'end_idx': 107, 'type': 'LOC'}",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을 지도자로 하는 남접으로 나뉘게 된다.","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, 'type': 'PER'}","{'word': '손병희', 'start_idx': 17, 'end_idx': 19, 'type': 'PER'}",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, 'type': 'ORG'}","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, 'type': 'PER'}",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이사장, 한국기원 총재를 맡고 있다.","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9, 'type': 'ORG'}","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': 25, 'type': 'ORG'}",no_relation,wikipedia


In [123]:
prc_list_sentence_unchanged

,id2,sentence2,subject_entity2,object_entity2,label2,source2
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': 23, 'type': 'ORG'}","{'word': '대안신당', 'start_idx': 14, 'end_idx': 17, 'type': 'ORG'}",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 24, 'type': 'ORG'}","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx': 41, 'type': 'ORG'}",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': 17, 'type': 'ORG'}","{'word': '박정부', 'start_idx': 22, 'end_idx': 24, 'type': 'PER'}",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_idx': 30, 'type': 'ORG'}","{'word': '1967', 'start_idx': 0, 'end_idx': 3, 'type': 'DAT'}",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등록 마감 결과, 강석호(3선·경북 영양·영덕·봉화·울진)-이장우(재선·대전 동구), 유기준(4선·부산 서구·동구)-박성중(초선·서울 서초을), 김선동(재선·서울 도봉을)-김종석(초선·비례), 심재철(5선·경기 안양시동안구을)-김재원(3선·경북 상주·군위·의성·청송) 등 4개 조가 등록했다고 밝혔다.","{'word': '유기준', 'start_idx': 93, 'end_idx': 95, 'type': 'PER'}","{'word': '부산 서구·동구', 'start_idx': 100, 'end_idx': 107, 'type': 'LOC'}",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을 지도자로 하는 남접으로 나뉘게 된다.","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, 'type': 'PER'}","{'word': '손병희', 'start_idx': 17, 'end_idx': 19, 'type': 'PER'}",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, 'type': 'ORG'}","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, 'type': 'PER'}",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이사장, 한국기원 총재를 맡고 있다.","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9, 'type': 'ORG'}","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': 25, 'type': 'ORG'}",no_relation,wikipedia


# 전처리가 되지 않은 문장 랜덤 확인

- 다음 셀을 반복 실행해서 전처리가 되지 않은 문장과 원본 문장을 확인해볼 수 있습니다.

In [151]:
random_id = random.choice(list(prc_list_sentence_unchanged.id2))
print("1. original id: ", org_list_sentence_unchanged.loc[random_id].id)
print("2. original sentence: ", org_list_sentence_unchanged.loc[random_id].sentence)
print("3. original subject_entity: ", org_list_sentence_unchanged.loc[random_id].subject_entity)
print("4. original object_entity: ", org_list_sentence_unchanged.loc[random_id].object_entity)
print("5. original label: ", org_list_sentence_unchanged.loc[random_id].label)
print("6. original source: ", org_list_sentence_unchanged.loc[random_id].source)
print("-"*120)
print("1. preprocessed id: ", prc_list_sentence_unchanged.loc[random_id].id2)
print("2. preprocessed sentence: ", prc_list_sentence_unchanged.loc[random_id].sentence2)
print("3. preprocessed subject_entity: ", prc_list_sentence_unchanged.loc[random_id].subject_entity2)
print("4. preprocessed object_entity: ", prc_list_sentence_unchanged.loc[random_id].object_entity2)
print("5. preprocessed label: ", prc_list_sentence_unchanged.loc[random_id].label2)
print("6. preprocessed source: ", prc_list_sentence_unchanged.loc[random_id].source2)

1. original id:  11409
2. original sentence:  결국 비스마르크는 1878년 2월 7일 교황 비오 9세가 사망한 후 레오 13세가 교황직을 승계하였다.
3. original subject_entity:  {'word': '비오 9세', 'start_idx': 25, 'end_idx': 29, 'type': 'PER'}
4. original object_entity:  {'word': '2월 7일', 'start_idx': 16, 'end_idx': 20, 'type': 'DAT'}
5. original label:  per:date_of_death
6. original source:  wikipedia
------------------------------------------------------------------------------------------------------------------------
1. preprocessed id:  11409
2. preprocessed sentence:  결국 비스마르크는 1878년 2월 7일 교황 비오 9세가 사망한 후 레오 13세가 교황직을 승계하였다.
3. preprocessed subject_entity:  {'word': '비오 9세', 'start_idx': 25, 'end_idx': 29, 'type': 'PER'}
4. preprocessed object_entity:  {'word': '2월 7일', 'start_idx': 16, 'end_idx': 20, 'type': 'DAT'}
5. preprocessed label:  per:date_of_death
6. preprocessed source:  wikipedia


---

- **전처리가 되지 않은 문장들이어도 코드 상의 오류로 인해 entity index가 변경되었을 수도 있습니다. 다음은 이를 확인합니다.**
- 이를 확인하기 위한 코드와 논리는 위의 전처리 된 문장들의 entity index를 확인하는 코드와 동일합니다.

- **다만 여기서는 전처리가 적용되지 않은 문장들의 index가 제대로 되어있는지 확인하는 것이기 때문에 전처리함수를 적용한 데이터셋 내의 entity index가 원본 데이터셋의 entity index와 동일한지도 추가로 확인해야 합니다. 왜냐하면 entity word가 한 문장 내에서 여러 번 등장할 때 ```fix_entity_index()``` 함수에 의해 의도치 않게 index가 변경되었을 수도 있기 때문입니다.**

In [126]:
count = 0
for row in tqdm(prc_list_sentence_unchanged.iterrows(), total = len(prc_list_sentence_unchanged)):
    idx = row[1].id2
    sts, subject_dict, object_dict = row[1].sentence2, eval(row[1].subject_entity2), eval(row[1].object_entity2)
    
    s_word, s_start, s_end = subject_dict['word'], subject_dict['start_idx'], subject_dict['end_idx']
    o_word, o_start, o_end = object_dict['word'], object_dict['start_idx'], object_dict['end_idx']
    
    if sts[s_start:s_end + 1] != s_word:
        count += 1
        print("wrong subject idx: " + idx)
    
    if sts[o_start:o_end + 1] != o_word:
        count += 1
        print("wrong object idx: " + idx)
    
    # 다음은 원본 데이터의 entity index와 전처리가 적용된 데이터의 entity index가 동일한지 확인합니다.
    org_subject_dict, org_object_dict = eval(org_list_sentence_unchanged.loc[idx].subject_entity), eval(org_list_sentence_unchanged.loc[idx].object_entity)
    
    org_s_start, org_s_end = org_subject_dict['start_idx'], org_subject_dict['end_idx']
    org_o_start, org_o_end = org_object_dict['start_idx'], org_object_dict['end_idx']
    
    if (s_start != org_s_start) or (s_end != org_s_end) or (o_start != org_o_start) or (o_end != org_o_end):
        count += 1
        print("different with original entity....idx: " + idx)
        
if count == 0:
    print("perfect preprocessing!")

  0%|          | 0/32171 [00:00<?, ?it/s]

perfect preprocessing!


---

# Test Data에 대한 전처리 수행 및 검증

- Train data에 대해 수행한 검증 내용을 동일하게 적용합니다.

In [105]:
processed_test_df = data_cleansing(test_df)
processed_test_df.columns = ['id2', 'sentence2', 'subject_entity2', 'object_entity2', 'label2', 'source2']

  0%|          | 0/7765 [00:00<?, ?it/s]

In [108]:
org_list_sentence_changed_test = test_df[test_df['sentence'] != processed_test_df['sentence2']]
prc_list_sentence_changed_test = processed_test_df[test_df['sentence'] != processed_test_df['sentence2']]

In [109]:
org_list_sentence_changed_test

,id,sentence,subject_entity,object_entity,label,source
800,800,"김복진(金復鎭, 1901년 11월 3일 (1901년 음력 9월 23일)(1901년 음력 9월 23일) - 1940년 8월 18일)은 일제 강점기의 조각가이며, 사회주의 계파의 진보 성향 문예운동을 이끈 지도자였다.","{'word': '김복진', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1901년 음력 9월 23일', 'start_idx': 23, 'end_idx': 37, 'type': 'DAT'}",100,wikipedia
898,898,"러시아에서, 《러시아 메신저》("""") 잡지에 레프 톨스토이의 《안나 카레니나》를 1873년부터 1877년까지 연재했고, 표도르 도스토옙스키의 《카라마조프의 형제》를 1879년부터 1880년까지 연재하였습니다.","{'word': '표도르 도스토옙스키', 'start_idx': 67, 'end_idx': 76, 'type': 'PER'}","{'word': '카라마조프의 형제', 'start_idx': 80, 'end_idx': 88, 'type': 'POH'}",100,wikipedia
1122,1122,"교수학습개발원이 마련한 ‘HNU Pre-School’ 프로그램은 교내 지식공유플랫폼인 Wisehub(학술정보원(도서관)(도서관)에 개설된 온라인 강의 콘텐츠를 활용하는 것으로, 외국어와 컴퓨터 자격증 강좌 등으로 진행된다.","{'word': '교수학습개발원', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}","{'word': 'HNU Pre-School', 'start_idx': 14, 'end_idx': 27, 'type': 'POH'}",100,wikitree
1192,1192,"《변신자동차 또봇》은 영실업과 레트로봇이 제작한 3D 로봇 애니메이션으로 또봇들의 활약상과 그의 파일럿들(차하나, 차두리, 권세모, 독고오공, 오혜라, 네옹(김네복)(김네복), 주딩요, 권리모, 국수호, 우표돌, 김녹원,)의 성장 이야기를 다루고 있다.","{'word': '차두리', 'start_idx': 64, 'end_idx': 66, 'type': 'PER'}","{'word': '또봇', 'start_idx': 41, 'end_idx': 42, 'type': 'POH'}",100,wikipedia
1374,1374,"또한, 고지서에 기재된 농협 가상계좌, 위택스(인터넷지로( 무안군ARS지방세(080-450-3838)(무안군ARS지방세(080-450-3838)(080-450-3838)를 이용하면 번거롭게 은행에 방문하지 않고도 다양한 방법으로 납부가 가능하다.","{'word': '농협', 'start_idx': 13, 'end_idx': 14, 'type': 'ORG'}","{'word': '무안군', 'start_idx': 57, 'end_idx': 59, 'type': 'LOC'}",100,wikitree
1761,1761,"파머는 그후 2011년 중반 미시건주 앤아버에서 촬영한 독립 로맨스 영화 《[[사랑과 전쟁 (2013년 영화)|사랑과 전쟁]》에 [[리엄 헴스워스]]와 같이 출연했다. 이 영화는 [[베트남 전쟁] 기간인 1960년대를 배경으로 했으며, 2013년 3월에 개봉되었다.","{'word': '파머', 'start_idx': 0, 'end_idx': 1, 'type': 'PER'}","{'word': '2013년', 'start_idx': 52, 'end_idx': 56, 'type': 'DAT'}",100,wikipedia
2149,2149,"므츠헤타에서 발견된 석비문은 ""카이사르의 친구"" 또한 """"로마가 사랑하는 이베리아의"" 왕으로 1세기의 통치자 미드라트 1세(58~56년)세를 증명한다.","{'word': '카이사르', 'start_idx': 17, 'end_idx': 20, 'type': 'PER'}","{'word': '미드라트 1세', 'start_idx': 61, 'end_idx': 67, 'type': 'PER'}",100,wikipedia
2363,2363,사업참여를 희망한 어촌계와 수리업체는 전남도청(및 해양수산과학원(ofsi.jeonnam.go.kr)(ofsi.jeonnam.go.kr) 누리집에 게재된 공고문을 참고해 해양수산과학원 각 지원에 방문하거나 팩스를 이용해 신청하면 된다.,"{'word': '해양수산과학원', 'start_idx': 28, 'end_idx': 34, 'type': 'ORG'}","{'word': 'ofsi.jeonnam.go.kr', 'start_idx': 56, 'end_idx': 73, 'type': 'POH'}",100,wikitree
2784,2784,"행사와 관련된 문의는 담양군 홈페이지(참고하거나 군청 문화체육과(061-380-2805)(061-380-2805), 담양문화회관(061-380-3465)으로 문의하면 된다.","{'word': '담양군', 'start_idx': 12, 'end_idx': 14, 'type': 'ORG'}","{'word': '061-380-2805', 'start_idx': 36, 'end_idx': 47, 'type': 'NOH'}",100,wikitree
3164,3164,"1792년 8월 10일 , 프랑스 혁명 정부는 국왕 루이 16세 체포권을 정지시켰다.","{'word': '정부', 'start_idx': 22, 'end_idx': 23, 'type': 'ORG'}","{'word': '프랑스 혁명', 'start_idx': 15, 'end_idx': 20, 'type': 'POH'}",100,wikipedia


In [110]:
prc_list_sentence_changed_test

,id2,sentence2,subject_entity2,object_entity2,label2,source2
800,800,"김복진(金復鎭, 1901년 11월 3일 (1901년 음력 9월 23일) - 1940년 8월 18일)은 일제 강점기의 조각가이며, 사회주의 계파의 진보 성향 문예운동을 이끈 지도자였다.","{'word': '김복진', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1901년 음력 9월 23일', 'start_idx': 23, 'end_idx': 37, 'type': 'DAT'}",100,wikipedia
898,898,"러시아에서, 《러시아 메신저》 잡지에 레프 톨스토이의 《안나 카레니나》를 1873년부터 1877년까지 연재했고, 표도르 도스토옙스키의 《카라마조프의 형제》를 1879년부터 1880년까지 연재하였습니다.","{'word': '표도르 도스토옙스키', 'start_idx': 63, 'end_idx': 72, 'type': 'PER'}","{'word': '카라마조프의 형제', 'start_idx': 76, 'end_idx': 84, 'type': 'POH'}",100,wikipedia
1122,1122,"교수학습개발원이 마련한 ‘HNU Pre-School’ 프로그램은 교내 지식공유플랫폼인 Wisehub(학술정보원(도서관)에 개설된 온라인 강의 콘텐츠를 활용하는 것으로, 외국어와 컴퓨터 자격증 강좌 등으로 진행된다.","{'word': '교수학습개발원', 'start_idx': 0, 'end_idx': 6, 'type': 'ORG'}","{'word': 'HNU Pre-School', 'start_idx': 14, 'end_idx': 27, 'type': 'POH'}",100,wikitree
1192,1192,"《변신자동차 또봇》은 영실업과 레트로봇이 제작한 3D 로봇 애니메이션으로 또봇들의 활약상과 그의 파일럿들(차하나, 차두리, 권세모, 독고오공, 오혜라, 네옹(김네복), 주딩요, 권리모, 국수호, 우표돌, 김녹원,)의 성장 이야기를 다루고 있다.","{'word': '차두리', 'start_idx': 64, 'end_idx': 66, 'type': 'PER'}","{'word': '또봇', 'start_idx': 41, 'end_idx': 42, 'type': 'POH'}",100,wikipedia
1374,1374,"또한, 고지서에 기재된 농협 가상계좌, 위택스(인터넷지로( 무안군ARS지방세(080-450-3838)(무안군ARS지방세(080-450-3838)를 이용하면 번거롭게 은행에 방문하지 않고도 다양한 방법으로 납부가 가능하다.","{'word': '농협', 'start_idx': 13, 'end_idx': 14, 'type': 'ORG'}","{'word': '무안군', 'start_idx': 57, 'end_idx': 59, 'type': 'LOC'}",100,wikitree
1761,1761,"파머는 그후 2011년 중반 미시건주 앤아버에서 촬영한 독립 로맨스 영화 《[사랑과 전쟁 (2013년 영화)|사랑과 전쟁]》에 [리엄 헴스워스]와 같이 출연했다. 이 영화는 [베트남 전쟁] 기간인 1960년대를 배경으로 했으며, 2013년 3월에 개봉되었다.","{'word': '파머', 'start_idx': 0, 'end_idx': 1, 'type': 'PER'}","{'word': '2013년', 'start_idx': 51, 'end_idx': 55, 'type': 'DAT'}",100,wikipedia
2149,2149,"므츠헤타에서 발견된 석비문은 ""카이사르의 친구"" 또한 로마가 사랑하는 이베리아의"" 왕으로 1세기의 통치자 미드라트 1세(58~56년)세를 증명한다.","{'word': '카이사르', 'start_idx': 17, 'end_idx': 20, 'type': 'PER'}","{'word': '미드라트 1세', 'start_idx': 59, 'end_idx': 65, 'type': 'PER'}",100,wikipedia
2363,2363,사업참여를 희망한 어촌계와 수리업체는 전남도청(및 해양수산과학원(ofsi.jeonnam.go.kr) 누리집에 게재된 공고문을 참고해 해양수산과학원 각 지원에 방문하거나 팩스를 이용해 신청하면 된다.,"{'word': '해양수산과학원', 'start_idx': 28, 'end_idx': 34, 'type': 'ORG'}","{'word': 'ofsi.jeonnam.go.kr', 'start_idx': 36, 'end_idx': 53, 'type': 'POH'}",100,wikitree
2784,2784,"행사와 관련된 문의는 담양군 홈페이지(참고하거나 군청 문화체육과(061-380-2805), 담양문화회관(061-380-3465)으로 문의하면 된다.","{'word': '담양군', 'start_idx': 12, 'end_idx': 14, 'type': 'ORG'}","{'word': '061-380-2805', 'start_idx': 36, 'end_idx': 47, 'type': 'NOH'}",100,wikitree
3164,3164,"1792년 8월 10일, 프랑스 혁명 정부는 국왕 루이 16세 체포권을 정지시켰다.","{'word': '정부', 'start_idx': 21, 'end_idx': 22, 'type': 'ORG'}","{'word': '프랑스 혁명', 'start_idx': 14, 'end_idx': 19, 'type': 'POH'}",100,wikipedia


# 전처리가 완료된 문장 랜덤 확인

- 다음 셀을 반복 실행하여 전처리가 완료된 문장을 랜덤으로 확인할 수 있습니다.

In [167]:
random_id = random.choice(list(prc_list_sentence_changed_test.id2))
print("1. original id: ", org_list_sentence_changed_test.loc[random_id].id)
print("2. original sentence: ", org_list_sentence_changed_test.loc[random_id].sentence)
print("3. original subject_entity: ", org_list_sentence_changed_test.loc[random_id].subject_entity)
print("4. original object_entity: ", org_list_sentence_changed_test.loc[random_id].object_entity)
print("5. original label: ", org_list_sentence_changed_test.loc[random_id].label)
print("6. original source: ", org_list_sentence_changed_test.loc[random_id].source)
print("-"*120)
print("1. preprocessed id: ", prc_list_sentence_changed_test.loc[random_id].id2)
print("2. preprocessed sentence: ", prc_list_sentence_changed_test.loc[random_id].sentence2)
print("3. preprocessed subject_entity: ", prc_list_sentence_changed_test.loc[random_id].subject_entity2)
print("4. preprocessed object_entity: ", prc_list_sentence_changed_test.loc[random_id].object_entity2)
print("5. preprocessed label: ", prc_list_sentence_changed_test.loc[random_id].label2)
print("6. preprocessed source: ", prc_list_sentence_changed_test.loc[random_id].source2)

1. original id:  3269
2. original sentence:  인현왕후 민씨(仁顯王后 閔氏, 1667년 5월 15일(음력 4월 23일)(음력 4월 23일) ~ 1701년 9월 16일(음력 8월 14일))는 조선 제19대 임금인 숙종의 계비이다.
3. original subject_entity:  {'word': '閔氏', 'start_idx': 13, 'end_idx': 14, 'type': 'PER'}
4. original object_entity:  {'word': '仁顯王后', 'start_idx': 8, 'end_idx': 11, 'type': 'PER'}
5. original label:  100
6. original source:  wikipedia
------------------------------------------------------------------------------------------------------------------------
1. preprocessed id:  3269
2. preprocessed sentence:  인현왕후 민씨(仁顯王后 閔氏, 1667년 5월 15일(음력 4월 23일) ~ 1701년 9월 16일(음력 8월 14일))는 조선 제19대 임금인 숙종의 계비이다.
3. preprocessed subject_entity:  {'word': '閔氏', 'start_idx': 13, 'end_idx': 14, 'type': 'PER'}
4. preprocessed object_entity:  {'word': '仁顯王后', 'start_idx': 8, 'end_idx': 11, 'type': 'PER'}
5. preprocessed label:  100
6. preprocessed source:  wikipedia


In [111]:
count = 0
for row in tqdm(prc_list_sentence_changed_test.iterrows(), total = len(prc_list_sentence_changed_test)):
    idx = row[1].id2
    sts, subject_dict, object_dict = row[1].sentence2, eval(row[1].subject_entity2), eval(row[1].object_entity2)
    
    s_word, s_start, s_end = subject_dict['word'], subject_dict['start_idx'], subject_dict['end_idx']
    o_word, o_start, o_end = object_dict['word'], object_dict['start_idx'], object_dict['end_idx']
    
    if sts[s_start:s_end + 1] != s_word:
        count += 1
        print("wrong subject idx: " + idx)
    
    if sts[o_start:o_end + 1] != o_word:
        count += 1
        print("wrong object idx: " + idx)

if count == 0:
    print("perfect preprocessing!")

  0%|          | 0/22 [00:00<?, ?it/s]

perfect preprocessing!


---

In [112]:
org_list_sentence_unchanged_test = test_df[test_df['sentence'] == processed_test_df['sentence2']]
prc_list_sentence_unchanged_test = processed_test_df[test_df['sentence'] == processed_test_df['sentence2']]

In [113]:
org_list_sentence_unchanged_test

,id,sentence,subject_entity,object_entity,label,source
0,0,지난 15일 MBC '탐사기획 스트레이트'가 이 사실을 보도했다.,"{'word': 'MBC', 'start_idx': 7, 'end_idx': 9, 'type': 'ORG'}","{'word': '탐사기획 스트레이트', 'start_idx': 12, 'end_idx': 21, 'type': 'ORG'}",100,wikitree
1,1,"사랑스러운 ‘프린세스 프링’의 이름은 봄의 공주님: Princess(s)Pring이란 뜻으로 탄생, 부활, 청춘 등 여러 가지 의미를 담아 생일왕국의 공주님의 이름이 되었다.","{'word': '프린세스 프링', 'start_idx': 7, 'end_idx': 13, 'type': 'PER'}","{'word': '공주', 'start_idx': 84, 'end_idx': 85, 'type': 'POH'}",100,wikipedia
2,2,"한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 1995년 6.25 특집극으로 편성될 예정이었는데 제작비 문제 때문에 연출자 고석만 PD(당시 단막극 팀장)가 MBC에 지원을 요청했지만 거절당한 데다 MBC가 경찰과의 관계 약화를 염려하여 제작을 달갑지 않게 여겼고 이 과정에서 연출자가 1995년 3월 29일 MBC에 사표를 제출한 후 프리랜서를 선언하여 편성이 무산됐으며 고석만 PD는 의 제작과 관련한 MBC와의 마찰 외에도 1994년 10월 ""평프로듀서로 일하고 싶다""며 당시 맡고 있던 단막극 책임PD(종합병원 전원일기 등등) 보직 사퇴의사를 밝혔지만 MBC 측의 반대로 무산되자 프리랜서를 선언했다.","{'word': '경찰', 'start_idx': 121, 'end_idx': 122, 'type': 'ORG'}","{'word': '1995년', 'start_idx': 31, 'end_idx': 35, 'type': 'DAT'}",100,wikipedia
3,3,"정창손은 김질과 같이 대궐로 달려가 고변하며 '신은 실로 모르고 김질만 혼자 참여하였는데, 김질의 죄는 만번 죽어 마땅합니다.'라고 하니, 세조가 특별히 김질을 사면하였다.","{'word': '세조', 'start_idx': 78, 'end_idx': 79, 'type': 'PER'}","{'word': '정창손', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}",100,wikipedia
4,4,"당시 민주당 이진련 시의원은 권영진 시장에게 ""긴급 생계자금을 왜 현금으로 지원하지 않느냐""고 따졌다.","{'word': '민주당', 'start_idx': 3, 'end_idx': 5, 'type': 'ORG'}","{'word': '권영진', 'start_idx': 16, 'end_idx': 18, 'type': 'PER'}",100,wikitree
...,...,...,...,...,...,...
7760,7760,코로나19 방역 조치의 일환으로 국민의 움직임을 통제하려는 정부의 시도를 이탈리아 국민이 어느 정도까지 수용할지도 의문이다.,"{'word': '정부', 'start_idx': 33, 'end_idx': 34, 'type': 'ORG'}","{'word': '이탈리아', 'start_idx': 41, 'end_idx': 44, 'type': 'LOC'}",100,wikitree
7761,7761,선 연구원은 “위식도역류질환치료제인 케이캡이 92억원 판매되면서 2019년 연간 328억원 매출을 기록했다”라며 “올해 약 500억원 이상의 매출 달성은 충분히 가능해 보인다. 기존 품목의 고른 성장과 신규 품목의 고성장세에 힘입어 올해 종근당의 탑라인은 전년대비 8% 이상 high single digit 성장률을 기록할 수 있을 것으로 보인다”라고 언급했다.,"{'word': '종근당', 'start_idx': 133, 'end_idx': 135, 'type': 'ORG'}","{'word': '전년', 'start_idx': 143, 'end_idx': 144, 'type': 'DAT'}",100,wikitree
7762,7762,"한국전기안전공사(사장 조성완)는 8월 1일부로, 3급 간부직원에 대한 승진·이동 인사를 아래와 같이 발령하였다.","{'word': '한국전기안전공사', 'start_idx': 0, 'end_idx': 7, 'type': 'ORG'}","{'word': '조성완', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}",100,wikitree
7763,7763,1987년 B. 슈나이더(B. Schneider)에 의해 만들어졌다.,"{'word': 'B. 슈나이더', 'start_idx': 6, 'end_idx': 12, 'type': 'PER'}","{'word': '1987년', 'start_idx': 0, 'end_idx': 4, 'type': 'DAT'}",100,wikipedia


In [114]:
prc_list_sentence_unchanged_test

,id2,sentence2,subject_entity2,object_entity2,label2,source2
0,0,지난 15일 MBC '탐사기획 스트레이트'가 이 사실을 보도했다.,"{'word': 'MBC', 'start_idx': 7, 'end_idx': 9, 'type': 'ORG'}","{'word': '탐사기획 스트레이트', 'start_idx': 12, 'end_idx': 21, 'type': 'ORG'}",100,wikitree
1,1,"사랑스러운 ‘프린세스 프링’의 이름은 봄의 공주님: Princess(s)Pring이란 뜻으로 탄생, 부활, 청춘 등 여러 가지 의미를 담아 생일왕국의 공주님의 이름이 되었다.","{'word': '프린세스 프링', 'start_idx': 7, 'end_idx': 13, 'type': 'PER'}","{'word': '공주', 'start_idx': 84, 'end_idx': 85, 'type': 'POH'}",100,wikipedia
2,2,"한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 1995년 6.25 특집극으로 편성될 예정이었는데 제작비 문제 때문에 연출자 고석만 PD(당시 단막극 팀장)가 MBC에 지원을 요청했지만 거절당한 데다 MBC가 경찰과의 관계 약화를 염려하여 제작을 달갑지 않게 여겼고 이 과정에서 연출자가 1995년 3월 29일 MBC에 사표를 제출한 후 프리랜서를 선언하여 편성이 무산됐으며 고석만 PD는 의 제작과 관련한 MBC와의 마찰 외에도 1994년 10월 ""평프로듀서로 일하고 싶다""며 당시 맡고 있던 단막극 책임PD(종합병원 전원일기 등등) 보직 사퇴의사를 밝혔지만 MBC 측의 반대로 무산되자 프리랜서를 선언했다.","{'word': '경찰', 'start_idx': 121, 'end_idx': 122, 'type': 'ORG'}","{'word': '1995년', 'start_idx': 31, 'end_idx': 35, 'type': 'DAT'}",100,wikipedia
3,3,"정창손은 김질과 같이 대궐로 달려가 고변하며 '신은 실로 모르고 김질만 혼자 참여하였는데, 김질의 죄는 만번 죽어 마땅합니다.'라고 하니, 세조가 특별히 김질을 사면하였다.","{'word': '세조', 'start_idx': 78, 'end_idx': 79, 'type': 'PER'}","{'word': '정창손', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}",100,wikipedia
4,4,"당시 민주당 이진련 시의원은 권영진 시장에게 ""긴급 생계자금을 왜 현금으로 지원하지 않느냐""고 따졌다.","{'word': '민주당', 'start_idx': 3, 'end_idx': 5, 'type': 'ORG'}","{'word': '권영진', 'start_idx': 16, 'end_idx': 18, 'type': 'PER'}",100,wikitree
...,...,...,...,...,...,...
7760,7760,코로나19 방역 조치의 일환으로 국민의 움직임을 통제하려는 정부의 시도를 이탈리아 국민이 어느 정도까지 수용할지도 의문이다.,"{'word': '정부', 'start_idx': 33, 'end_idx': 34, 'type': 'ORG'}","{'word': '이탈리아', 'start_idx': 41, 'end_idx': 44, 'type': 'LOC'}",100,wikitree
7761,7761,선 연구원은 “위식도역류질환치료제인 케이캡이 92억원 판매되면서 2019년 연간 328억원 매출을 기록했다”라며 “올해 약 500억원 이상의 매출 달성은 충분히 가능해 보인다. 기존 품목의 고른 성장과 신규 품목의 고성장세에 힘입어 올해 종근당의 탑라인은 전년대비 8% 이상 high single digit 성장률을 기록할 수 있을 것으로 보인다”라고 언급했다.,"{'word': '종근당', 'start_idx': 133, 'end_idx': 135, 'type': 'ORG'}","{'word': '전년', 'start_idx': 143, 'end_idx': 144, 'type': 'DAT'}",100,wikitree
7762,7762,"한국전기안전공사(사장 조성완)는 8월 1일부로, 3급 간부직원에 대한 승진·이동 인사를 아래와 같이 발령하였다.","{'word': '한국전기안전공사', 'start_idx': 0, 'end_idx': 7, 'type': 'ORG'}","{'word': '조성완', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}",100,wikitree
7763,7763,1987년 B. 슈나이더(B. Schneider)에 의해 만들어졌다.,"{'word': 'B. 슈나이더', 'start_idx': 6, 'end_idx': 12, 'type': 'PER'}","{'word': '1987년', 'start_idx': 0, 'end_idx': 4, 'type': 'DAT'}",100,wikipedia


# 전처리가 적용되지 않은 문장 랜덤 확인

- 다음 셀을 반복 실행하여 전처리가 적용되지 않은 문장을 확인할 수 있습니다.

In [169]:
random_id = random.choice(list(prc_list_sentence_unchanged_test.id2))
print("1. original id: ", org_list_sentence_unchanged_test.loc[random_id].id)
print("2. original sentence: ", org_list_sentence_unchanged_test.loc[random_id].sentence)
print("3. original subject_entity: ", org_list_sentence_unchanged_test.loc[random_id].subject_entity)
print("4. original object_entity: ", org_list_sentence_unchanged_test.loc[random_id].object_entity)
print("5. original label: ", org_list_sentence_unchanged_test.loc[random_id].label)
print("6. original source: ", org_list_sentence_unchanged_test.loc[random_id].source)
print("-"*120)
print("1. preprocessed id: ", prc_list_sentence_unchanged_test.loc[random_id].id2)
print("2. preprocessed sentence: ", prc_list_sentence_unchanged_test.loc[random_id].sentence2)
print("3. preprocessed subject_entity: ", prc_list_sentence_unchanged_test.loc[random_id].subject_entity2)
print("4. preprocessed object_entity: ", prc_list_sentence_unchanged_test.loc[random_id].object_entity2)
print("5. preprocessed label: ", prc_list_sentence_unchanged_test.loc[random_id].label2)
print("6. preprocessed source: ", prc_list_sentence_unchanged_test.loc[random_id].source2)

1. original id:  6428
2. original sentence:  이날 협약으로 광산구는 광주신보에 1억 원을 출연하고, 재단은 출연금의 15배인 15억원을 소상공인 대출 보증재원으로 운용해, 광산구 소상공인에게 각각 최고 2,000만원 이내 경영자금 지원을 제공할 예정.
3. original subject_entity:  {'word': '광주신보', 'start_idx': 13, 'end_idx': 16, 'type': 'ORG'}
4. original object_entity:  {'word': '광산구', 'start_idx': 8, 'end_idx': 10, 'type': 'ORG'}
5. original label:  100
6. original source:  wikitree
------------------------------------------------------------------------------------------------------------------------
1. preprocessed id:  6428
2. preprocessed sentence:  이날 협약으로 광산구는 광주신보에 1억 원을 출연하고, 재단은 출연금의 15배인 15억원을 소상공인 대출 보증재원으로 운용해, 광산구 소상공인에게 각각 최고 2,000만원 이내 경영자금 지원을 제공할 예정.
3. preprocessed subject_entity:  {'word': '광주신보', 'start_idx': 13, 'end_idx': 16, 'type': 'ORG'}
4. preprocessed object_entity:  {'word': '광산구', 'start_idx': 8, 'end_idx': 10, 'type': 'ORG'}
5. preprocessed label:  100
6. preprocessed source:  wikitree


In [127]:
count = 0
for row in tqdm(prc_list_sentence_unchanged_test.iterrows(), total = len(prc_list_sentence_unchanged_test)):
    idx = row[1].id2
    sts, subject_dict, object_dict = row[1].sentence2, eval(row[1].subject_entity2), eval(row[1].object_entity2)
    
    s_word, s_start, s_end = subject_dict['word'], subject_dict['start_idx'], subject_dict['end_idx']
    o_word, o_start, o_end = object_dict['word'], object_dict['start_idx'], object_dict['end_idx']
    
    if sts[s_start:s_end + 1] != s_word:
        count += 1
        print("wrong subject idx: " + idx)
    
    if sts[o_start:o_end + 1] != o_word:
        count += 1
        print("wrong object idx: " + idx)

    # 다음은 원본 데이터의 entity index와 전처리가 적용된 데이터의 entity index가 동일한지 확인합니다.
    org_subject_dict, org_object_dict = eval(org_list_sentence_unchanged_test.loc[idx].subject_entity), eval(org_list_sentence_unchanged_test.loc[idx].object_entity)
    
    org_s_start, org_s_end = org_subject_dict['start_idx'], org_subject_dict['end_idx']
    org_o_start, org_o_end = org_object_dict['start_idx'], org_object_dict['end_idx']
    
    if (s_start != org_s_start) or (s_end != org_s_end) or (o_start != org_o_start) or (o_end != org_o_end):
        count += 1
        print("different with original entity....idx: " + idx)
if count == 0:
    print("perfect preprocessing!")

  0%|          | 0/7743 [00:00<?, ?it/s]

perfect preprocessing!
